# Recommender model based on LightFM
## An attempt to break a record on Kaggle (AUC score)
A link to competition https://www.kaggle.com/c/recommendationsv4/overview

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix as sp
import time
import matplotlib
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm.cross_validation import random_train_test_split 

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

/home/pigidser/anaconda3/envs/recsys/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/pigidser/anaconda3/envs/recsys/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.head(3)

,overall,verified,reviewTime,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image,userid,itemid,rating
0,5.0,True,"10 4, 2016",B01CPNIEQG,Heather,These are my FAVORITE spices in my collection....,Must Add to your Spice kitchen!,1475539200,NaN,NaN,NaN,102179,37138,1.0
1,5.0,True,"03 1, 2016",B006F63M8U,Linda Odom,Add A package to my Coffee and it makes a good...,Milk Chocolate Swiis MIss Hot Cocoa Mix,1456790400,NaN,{'Size:': ' 60-0.73 oz Envelopes'},NaN,3625,17322,1.0
2,5.0,True,"06 26, 2016",B00112O8NG,DesertBlossom,"I love the Torani syrups, but not the prices o...","Love these sugar free syrups, but didn't love ...",1466899200,28,NaN,NaN,39495,5600,1.0


In [4]:
%%time
# Building the Model
dataset = Dataset()
dataset.fit(train.userid.values,
            train.asin.values)
dataset.fit_partial(test.userid.values,
                    test.asin.values)

num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 127496, num_items 41320.
CPU times: user 1.31 s, sys: 20.1 ms, total: 1.33 s
Wall time: 1.33 s


In [5]:
%%time
(interactions, weights) = dataset.build_interactions(
    [tuple(row) for row in train[['userid','asin','rating']].values])

print(repr(interactions),repr(weights))

<127496x41320 sparse matrix of type '<class 'numpy.int32'>'
	with 857895 stored elements in COOrdinate format> <127496x41320 sparse matrix of type '<class 'numpy.float32'>'
	with 857895 stored elements in COOrdinate format>
CPU times: user 2.47 s, sys: 51.9 ms, total: 2.52 s
Wall time: 2.52 s


In [6]:
%%time
# Training the Model

NUM_THREADS = 8 #число потоков
NUM_COMPONENTS = 200 #число параметров вектора 
NUM_EPOCHS = 20 #число эпох обучения

model = LightFM(learning_rate=0.1, loss='logistic', # 'logistic', 'warp', 'bpr', 'warp-kos'
                no_components=NUM_COMPONENTS)
model = model.fit(weights, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS, verbose=True)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
CPU times: user 1min 18s, sys: 3.57 s, total: 1min 22s
Wall time: 21.3 s


In [7]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()
test_user_ids = np.array(list(map(user_id_map.get, list(test.userid.values))))
test_item_ids = np.array(list(map(item_id_map.get, list(test.asin.values))))

preds = model.predict(test_user_ids,
                      test_item_ids,
                      num_threads=8)

normalized_preds = (preds - preds.min())/(preds - preds.min()).max()
submission['rating']= preds
submission.to_csv('submission.csv', index=None)

In [8]:
!kaggle competitions submit -c recommendationsv4 -f submission.csv -m "no item_features, logistic with lr=.1 and 20 epochs"

100%|██████████████████████████████████████| 6.90M/6.90M [00:04<00:00, 1.53MB/s]
Successfully submitted to [SF-DST] Recommendation Challenge v4